In [0]:
# %pip install --quiet openai

In [0]:
import os, random, re
from typing import Optional, Dict
from pyspark.sql import Row
from openai import OpenAI

# --- Silence MLflow tracing warnings completely ---
import warnings
import logging
import mlflow

# Disable MLflow autologging and tracing
mlflow.autolog(disable=True)

# Turn off loggers for MLflow and tracing
logging.getLogger("mlflow").setLevel(logging.CRITICAL)
logging.getLogger("mlflow.tracing").setLevel(logging.CRITICAL)
logging.getLogger("mlflow.tracing.export.mlflow_v3").setLevel(logging.CRITICAL)

# Suppress all warnings containing 'mlflow.tracing'
warnings.filterwarnings("ignore", message=".*mlflow.tracing.*", category=UserWarning)
warnings.filterwarnings("ignore", message=".*trace to MLflow backend.*", category=UserWarning)
# ---------------------------------------------------

In [0]:
# 1) Auth —— put your key in env or use Databricks Secrets
os.environ["OPENAI_API_KEY"] = ""   
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [0]:

# 2) Fetch context for a single user (adjust fields if you add more)
def fetch_prompt_context(user_id: int) -> Optional[Dict]:
    row: Optional[Row] = (spark.table("instacart.gold.customer_segments")
                          .where(f"user_id = {int(user_id)}")
                          .select(
                              "user_id","temporal_group","engagement_group",
                              "preferred_window","dominant_dow","dominant_hour",
                              "mean_days_between_orders","last_gap_days",
                              "top_product_1","top_product_2","top_product_3"
                          )
                          .limit(1)
                          .collect()[0] if spark.table("instacart.gold.customer_segments").where(f"user_id = {int(user_id)}").count() > 0 else None)
    if not row:
        return None
    d = row.asDict()
    # small helpers
    dow_map = ["Sun","Mon","Tue","Wed","Thu","Fri","Sat"]
    d["dominant_dow_label"] = dow_map[int(d["dominant_dow"])]
    d["top_products"] = [p for p in [d["top_product_1"], d["top_product_2"], d["top_product_3"]] if p]
    return d

# 3) Build prompt (EDIT THESE SECTIONS freely)
def build_prompt(ctx: Dict, style: str = "friendly") -> str:
    return f"""
[ROLE]
You are a retention copywriter. Write a concise, on-brand push notification.

[GOAL]
Nudge the user to reorder around their usual time window with 1–2 favorite items.

[CONSTRAINTS]
- 18–28 words, plain language, no emojis.
- Include exactly one product if available.
- Match tone to engagement group. Be respectful if at-risk.
- Do NOT fabricate data.

[USER CONTEXT]
user_id: {ctx['user_id']}
temporal_group: {ctx['temporal_group']}
engagement_group: {ctx['engagement_group']}
preferred_window: {ctx['preferred_window']}
dominant_dow: {ctx['dominant_dow_label']}
dominant_hour: {int(ctx['dominant_hour'])}:00
mean_gap_days: {round(float(ctx['mean_days_between_orders']),1)}
last_gap_days: {round(float(ctx['last_gap_days']),1)}
fav_products: {", ".join(ctx["top_products"]) if ctx["top_products"] else "N/A"}

[STYLE]
tone: {style}
brand: practical, helpful, time-aware

[OUTPUT]
One push notification line. No preamble, no quotes.
""".strip()

# 4) Generate with OpenAI (Responses API). Swap model/params as you like.
def generate_notification(user_id: int,
                          model: str = "gpt-4o-mini",
                          temperature: float = 0.7,
                          style: str = "friendly") -> str:
    ctx = fetch_prompt_context(user_id)
    if not ctx:
        return f"[no-context] user_id {user_id} not found in customer_segments."
    prompt = build_prompt(ctx, style=style)

    resp = client.responses.create(
        model=model,
        input=prompt,
        temperature=temperature,
        max_output_tokens=60
    )
    return resp.output_text.strip()

In [0]:


# 1) Reassuring tone, soft CTA, no incentive (at-risk friendly)
print(generate_notification(
    user_id=47,                 # Late-Night • At-Risk Regular (from your sample)
    model="gpt-4o-mini",
    temperature=0.6,
    style="reassuring",
    cta_mode="soft",
    include_incentive=False,
    min_words=16, max_words=26, n_variants=6
))
print()

# 2) Energetic tone, standard CTA, include incentive
print(generate_notification(
    user_id=27,                 # Weekly Shoppers (active)
    model="gpt-4o-mini",
    temperature=0.8,
    style="energetic",
    cta_mode="standard",
    include_incentive=True,
    min_words=18, max_words=24, n_variants=7
))
print()

# 3) Exclusive tone, urgent CTA (time-sensitive push)
print(generate_notification(
    user_id=22,                 # Evening Weekend
    model="gpt-4o-mini",
    temperature=0.7,
    style="exclusive",
    cta_mode="urgent",
    include_incentive=False,
    min_words=14, max_words=22, n_variants=6
))
print()

# 4) Minimal tone, standard CTA (short + direct)
print(generate_notification(
    user_id=53,                 # Morning Weekday
    model="gpt-4o-mini",
    temperature=0.5,
    style="minimal",
    cta_mode="standard",
    include_incentive=False,
    min_words=12, max_words=18, n_variants=8
))
print()

# 5) Seasonal tone, urgent CTA, include incentive (campaign-style)
print(generate_notification(
    user_id=76,                 # Morning Weekday
    model="gpt-4o-mini",
    temperature=0.9,
    style="seasonal",
    cta_mode="urgent",
    include_incentive=True,
    min_words=18, max_words=28, n_variants=6
))
